# Homework 5
任课老师：胡俊峰

负责助教：陈滨琪、杨礼铭

截止日期：3月15日12点

## 0.1 请阅读以下代码，理解代码功能，分析代码运行逻辑

下面的代码展示了使用多线程与Queue实现监听函数与后台工作函数分离的案例，运行时间约为50秒。该代码使用dict记录了等待销毁(join)的线程信息。该代码存在一个小缺憾：如果runtime_thread线程被销毁，则在其被销毁时还在运行的工作线程需要等到下次收到数据后才能被销毁(join)。可以简单思考一下这个问题的解决方法。

In [11]:
# 多线程实现监听函数与后台工作函数分离
from time import sleep, time
import random
import threading
from queue import Queue


data_queue = Queue()
thread_working = False
thre = None
thread_dict = {}
finish_thread_names = Queue()


def process_data(data):
    global finish_thread_names
    print('-----inside single worker-----')
    sleep_time = 4 + random.randint(-3, 3)
    print(
        f'begin processing: 【 {str(data)} 】 work time set = {sleep_time} second')
    bg_time = time()
    sleep(sleep_time)
    print(
        f'finish processing: 【 {str(data)} 】 work time set = {sleep_time} second, true work time = {str(time()-bg_time)[:5]} second')

    finish_thread_names.put(threading.current_thread().name)

    return


def runtime_thread():
    global data_queue, thread_working, thread_dict, finish_thread_names
    print('\n----------------------------inside working runtime----------------------\n')
    while not data_queue.empty():
        x = data_queue.get()
        tmp_thread = threading.Thread(target=process_data, args=[x])
        thread_name = tmp_thread.name
        thread_dict[thread_name] = tmp_thread
        tmp_thread.start()

        sleep(1)  # 每秒最多启动一个工作线程

        while not finish_thread_names.empty():

            x = finish_thread_names.get()
            died_thread = thread_dict[x]
            del thread_dict[x]
            died_thread.join()
            print(f'thread_join, now die threads = {x}')

    print('\n----------------------------out from working runtime----------------------\n')
    thread_working = False

    return


def send_notification(data):
    global data_queue, thre, thread_working, thread_dict, finish_thread_names
    data_queue.put(data)
    if not thre:
        thre = threading.Thread(target=runtime_thread)
        thread_working = True
        thre.start()
    elif not thread_working:

        while not finish_thread_names.empty():
            x = finish_thread_names.get()

            died_thread = thread_dict[x]
            del thread_dict[x]

            died_thread.join()
            print(f'thread_join, now die threads = {x}')

        thre.join()
        thre = threading.Thread(target=runtime_thread)
        thread_working = True
        thre.start()

    return


if __name__ == '__main__':
    data = 1
    while data <= 10:
        send_notification(data)
        print(f'send data 【 {data} 】 to queue')
        sleep(random.uniform(0, 1.7))
        data += 1

    print('\n\n-------------------------------All data has been put into the queue!!----------------------\n\n')
    # 感觉放一个销毁的while循环在这里是不是就可以解决
    thre.join()
    while not finish_thread_names.empty():
            x = finish_thread_names.get()

            died_thread = thread_dict[x]
            del thread_dict[x]

            died_thread.join()
            print(f'thread_join, now die threads = {x}')
    # 好像不行 是不是要记录工作线程的开始 然后最后join所有没结束的线程




----------------------------inside working runtime----------------------
send data 【 1 】 to queue

-----inside single worker-----
begin processing: 【 1 】 work time set = 2 second
send data 【 2 】 to queue
-----inside single worker-----
begin processing: 【 2 】 work time set = 3 second
send data 【 3 】 to queue
finish processing: 【 1 】 work time set = 2 second, true work time = 2.158 secondsend data 【 4 】 to queue
-----inside single worker-----
begin processing: 【 3 】 work time set = 7 second

thread_join, now die threads = Thread-165 (process_data)
-----inside single worker-----
begin processing: 【 4 】 work time set = 1 second
send data 【 5 】 to queue
finish processing: 【 2 】 work time set = 3 second, true work time = 3.002 second
finish processing: 【 4 】 work time set = 1 second, true work time = 1.005 second
thread_join, now die threads = Thread-166 (process_data)
thread_join, now die threads = Thread-168 (process_data)
-----inside single worker-----
begin processing: 【 5 】 work time s

## 0.2 递归生成器的用法

In [12]:
nested = [[[1], 2], 3, 4, [5, [[6, 7]]]]

def flatten(nested):
    for item in nested:
        if isinstance(item, int):
            yield item
        else:
            for element in flatten(item):
                yield element

for i in flatten(nested):
    print(i)

1
2
3
4
5
6
7


# 1 多线程初步
请根据已有的代码框架对于多线程进行初步的实现。使用threading模块实现多线程并行执行，分别执行任务一、任务二、任务三(任务执行的顺序不做要求)。(2分)

In [4]:
import threading
import time


def task(i):
    start = time.time()
    print(f'Threading: {threading.current_thread().name} - Task{i} start!')
    time.sleep(1)
    end = time.time()
    print(f'Threading: {threading.current_thread().name} - Task{i} cost {end-start} seconds!\n',end='')


# 串行执行三个任务
print('串行执行:')
start = time.time()
for i in range(1, 4):
    task(i)
end = time.time()
print(f'Cost {end - start} seconds!')

# 并行执行三个任务
print('多线程并行执行:')
start = time.time()
thread_list = []
# TODO: 请在下面的代码中添加代码，实现多线程并行执行三个任务
for i in range(1, 4):
    thread_list.append(t:=threading.Thread(target=task, args=(i,)))
    t.start()
for i in thread_list:
    i.join()
end = time.time()
print(f'Cost {end - start} seconds!')


串行执行:
Threading: MainThread - Task1 start!
Threading: MainThread - Task1 cost 1.0021772384643555 seconds!
Threading: MainThread - Task2 start!
Threading: MainThread - Task2 cost 1.008155345916748 seconds!
Threading: MainThread - Task3 start!
Threading: MainThread - Task3 cost 1.0009186267852783 seconds!
Cost 3.011251211166382 seconds!
多线程并行执行:
Threading: Thread-5 (task) - Task1 start!
Threading: Thread-6 (task) - Task2 start!
Threading: Thread-7 (task) - Task3 start!
Threading: Thread-5 (task) - Task1 cost 1.0019409656524658 seconds!
Threading: Thread-7 (task) - Task3 cost 1.0029559135437012 seconds!
Threading: Thread-6 (task) - Task2 cost 1.0029559135437012 seconds!
Cost 1.0029559135437012 seconds!


# 2 yield迭代器
模仿课件中的例子，使用yield实现一个有隐含功能的迭代器：初始迭代器中的数组为空，可以通过send来更新迭代器中的list，当send一个list时，返回list中的最大值，若send的list为空时返回None。当调用next方法时，默认返list中的下一个数，每次使用send更新list后，下一次调用next输出的是更新后list的第一个数，当数组为空或多次迭代数组越界时也返回None(可以参考样例理解功能要求)。(2分)

In [3]:
def gen_cal():
    num_list = []
    # TODO: 请在下面的代码中添加代码，使用yield实现有隐含功能的迭代器
    catch = []
    while True:
        if not num_list:
            catch = yield None
            num_list = catch
        elif catch and num_list != []:
            catch = yield max(num_list)
        elif not catch:
            catch = yield num_list.pop(0)
            if catch != None:
                num_list = catch
    
gc = gen_cal()
assert(next(gc) == None)  # 初始list为空输出None
assert(gc.send([2, 7, 4]) == 7)  # 传入[2,7,4]，输出最大值7
assert(next(gc) == 2)  # 调用next，输出第一个数2
assert(next(gc) == 7)  # 再次调用next，输出第二个数7
assert(gc.send([1, 2]) == 2)  # 传入[1,2]，输出最大值2
assert(next(gc) == 1)  # 调用next，输出第一个数1
assert(gc.send([]) == None)  # 传入空list，输出None


# 3 协程轮转调度
在课件轮转调度框架的基础上完成以下题目:
现在有Task1和Task2两个任务，这两个任务又可以划分为若干子任务，由于子任务之间存在依赖关系，因此需要按照一定的顺序执行。必须先执行等级较低的子任务，才可以执行等级较高的子任务，同一等级的子任务可能有很多个，可以将同一等级的任务看做相同的(即执行顺序不做要求)，每次只能执行一个子任务。当Task1或者Task2之一执行完成之后，将会由另一个任务单独执行直到也执行完毕。

每次执行完某个等级的子任务输出对应等级剩余子任务的信息，例如:Task1执行了等级2的子任务之后剩余1个等级2的子任务，则输出`Task1: level 2 subtask remains 1.`当所有子任务执行完毕后输出`Task1 finished!`(可以根据给定的样例输出理解题目要求)。(3分)

In [7]:
waiting_list = []
def worker1():
    # 等级1两个，等级2一个，等级3一个，等级4两个
    task = {1: 2, 2: 1, 3: 1, 4: 2}
    rank = 1
    while rank <= max(task.keys()):
        while task[rank] > 0:
            task[rank] -= 1
            print(f"Task1: level {rank} subtask remains {task[rank]}.")
            yield
        rank += 1
    print('Task1 finished!')


def worker2():
    # 等级1两个，等级2四个，等级3三个
    task = {1: 2, 2: 4, 3: 3}
    rank = 1
    #TODO: 请在下面的代码中添加代码，执行的顺序为等级1->等级2->等级3
    while rank <= max(task.keys()):
        while task[rank] > 0:
            task[rank] -= 1
            print(f"Task2: level {rank} subtask remains {task[rank]}.")
            yield
        rank += 1
    
    print('Task2 finished!')


class Handle(object):

    def __init__(self, gen):
        self.gen = gen

    def call(self):
        try:
            next(self.gen)
            waiting_list.append(self)
        except StopIteration:
            pass
            # print('test')
            # print(f"Task{self.gen.__name__[-1]} finished!")


def RoundRobin(*worker):
    waiting_list.extend(Handle(c) for c in worker)
    #TODO: 请在下面的代码中添加代码，实现RoundRobin调度
    while waiting_list:
        p = waiting_list.pop(0)
        p.call()
    print('All Task Finished!')


w1 = worker1()
w2 = worker2()
RoundRobin(w1, w2)


Task1: level 1 subtask remains 1.
Task2: level 1 subtask remains 1.
Task1: level 1 subtask remains 0.
Task2: level 1 subtask remains 0.
Task1: level 2 subtask remains 0.
Task2: level 2 subtask remains 3.
Task1: level 3 subtask remains 0.
Task2: level 2 subtask remains 2.
Task1: level 4 subtask remains 1.
Task2: level 2 subtask remains 1.
Task1: level 4 subtask remains 0.
Task2: level 2 subtask remains 0.
Task1 finished!
Task2: level 3 subtask remains 2.
Task2: level 3 subtask remains 1.
Task2: level 3 subtask remains 0.
Task2 finished!
All Task Finished!


# 4 async/await
worker_1只能完成第一类和第三类的任务，worker_2只能完成第二类的任务，请参考课件中的例子完成以下要求：worker如果在自己的任务队列中发现自己不能完成的任务，就将其交给另一个worker去完成(只需要修改任务队列，对任务执行的顺序不做要求，不考虑一个worker执行完所有任务之后，另一个worker又将任务交给他的情况)。(3分)

In [4]:
import asyncio


class Task():
    def __init__(self, type, time):
        self.type = type  # 任务类型
        self.time = time  # 任务耗时


task_list1 = [Task(1, 2), Task(1, 3), Task(2, 2), Task(3, 4)]
task_list2 = [Task(2, 1), Task(2, 3), Task(2, 4), Task(3, 2)]
interval = .1

# 完成任务时调用working函数
async def working(worker_id, task: Task):
    """working函数

    Args:
        worker_id : worker的编号，0或1
        task : 任务对象
    """
    print(
        f"worker {worker_id} is working on task {task.type} and cost {task.time} seconds!")
    await asyncio.sleep(task.time)


async def worker_1():
    # TODO: 请在下面的代码中添加代码，实现worker_1的执行过程
    for i in task_list1:
        if i.type in (1,3):
            await working(1, i)
        else:
            task_list2.append(i)


async def worker_2():
    # TODO: 请在下面的代码中添加代码，实现worker_2的执行过程
    for i in task_list2:
        if i.type in (2,):
            await working(2, i)
        else:
            task_list1.append(i)



async def main():
    # TODO: 请在下面的代码中添加代码，参考课件的例子补全main函数
    task1 = asyncio.create_task(worker_1())
    #task2 = asyncio.create_task(worker_2())
    print('--------------')
    await task1
    print('--------------')
    #await task2
print('test')
await main()


test
--------------
worker 1 is working on task 1 and cost 2 seconds!
worker 1 is working on task 1 and cost 3 seconds!
worker 1 is working on task 3 and cost 4 seconds!
--------------


In [2]:
import asyncio

async def f1():
    print("f1")
    await f2()

async def f2():
    print("f2")
    await f1()

async def main():
    await f1()
await main()

f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f2
f1
f

RecursionError: maximum recursion depth exceeded while calling a Python object